In [2]:
class Space:
    def __init__(self, *args):
        self.code = args[0]
        self.size = args[1]


class Group:
    def __init__(self, *args):
        self.id = args[0]
        self.size = args[1]


class Activity:
    def __init__(self, id, *args):
        self.id = id
        self.subject = args[0]
        self.teacher_id = args[1]
        self.group_ids = args[2]
        self.duration = args[3]

    def __repr__(self):
        return f"Activity(id={self.id}, subject={self.subject}, teacher_id={self.teacher_id}, group_ids={self.group_ids}, duration={self.duration})"


class Period:
    def __init__(self, *args):
        self.space = args[0]
        self.group = args[1]
        self.activity = args[2]

In [3]:
import json

# Load data from JSON file
with open('sliit_computing_dataset.json', 'r') as file:
    data = json.load(file)

# Create dictionaries to store instances
spaces_dict = {}
groups_dict = {}
activities_dict = {}
slots = []
# Populate the dictionaries with data from the JSON file
for space in data['spaces']:
    spaces_dict[space['code']] = Space(space['code'], space['capacity'])

for group in data['years']:
    groups_dict[group['id']] = Group(group['id'], group['size'])

for activity in data['activities']:
    activities_dict[activity['code']] = Activity(
        activity['code'], activity['subject'], activity['teacher_ids'][0], activity['subgroup_ids'], activity['duration'])

for day in ["MON", "TUE", "WED", "THU", "FRI"]:
    for id in range(1, 9):
        slots.append(day+str(id))
# Print the dictionaries to verify
print(spaces_dict)
print(groups_dict)
print(activities_dict)
print(slots)

{'LH401': <__main__.Space object at 0x000001ECA2C3C500>, 'LH501': <__main__.Space object at 0x000001ECA3719730>, 'LAB501': <__main__.Space object at 0x000001ECA6075580>, 'LAB502': <__main__.Space object at 0x000001ECA6074E00>}
{'Y1S1.1': <__main__.Group object at 0x000001ECA6076C90>, 'Y1S1.2': <__main__.Group object at 0x000001ECA60749E0>, 'Y1S1.3': <__main__.Group object at 0x000001ECA60751C0>, 'Y1S1.4': <__main__.Group object at 0x000001ECA6076960>, 'Y1S1.5': <__main__.Group object at 0x000001ECA6074C20>, 'Y1S2.1': <__main__.Group object at 0x000001ECA6076BD0>, 'Y1S2.2': <__main__.Group object at 0x000001ECA6075490>, 'Y1S2.3': <__main__.Group object at 0x000001ECA6075790>, 'Y1S2.4': <__main__.Group object at 0x000001ECA60745F0>, 'Y1S2.5': <__main__.Group object at 0x000001ECA6075460>, 'Y2S1.1': <__main__.Group object at 0x000001ECA6077440>, 'Y2S1.2': <__main__.Group object at 0x000001ECA6075250>, 'Y2S1.3': <__main__.Group object at 0x000001ECA6076750>, 'Y2S1.4': <__main__.Group objec

In [18]:
import random


def fitness(solution):
    room_overbooking = 0
    slot_conflicts = 0
    prof_conflicts = 0

    slot_assignments = {}
    prof_assignments = {}

    # Evaluate room overbooking and slot conflicts
    for entry in solution:
        course_id, student_group, room_id, valid_slots, professor = entry
        course = activities_dict[course_id]
        room = spaces_dict[room_id]

        # Room overbooking
        if room.size < get_classsize(course):
            room_overbooking += 1

        # Slot conflicts (same student in multiple classes at the same time)
        for slot_id in valid_slots:
            if slot_id not in slot_assignments:
                slot_assignments[slot_id] = []
            slot_assignments[slot_id].append((student_group, course_id))

        # Professor conflicts (same professor teaching multiple courses at the same time)
        if course.teacher_id not in prof_assignments:
            prof_assignments[course.teacher_id] = []
        prof_assignments[course.teacher_id].append(slot_id)

    # Calculate slot conflicts
    for slot_id, assignments in slot_assignments.items():
        student_groups_in_slot = {}
        for student_groups, course_id in assignments:
            for student_group in student_groups:
                if student_group in student_groups_in_slot:
                    slot_conflicts += 1  # Conflict when a student is assigned to two courses at the same time
                else:
                    student_groups_in_slot[student_group] = course_id

    # Calculate professor conflicts
    for professor, assigned_slots in prof_assignments.items():
        if len(assigned_slots) > len(set(assigned_slots)):  # Duplicate slots means conflict
            prof_conflicts += 1

    return room_overbooking, slot_conflicts, prof_conflicts


def get_classsize(activity: Activity) -> int:
    classsize = 0
    for id in activity.group_ids:
        classsize += groups_dict[id].size
    return classsize

# Updated create_population function


def create_population(pop_size):
    population = []
    for _ in range(pop_size):
        solution = []
        room_slot_assignments = {space.code: set()
                                 # Track slots per room
                                 for space in spaces_dict.values()}
        prof_slot_assignments = {prof: set() for prof in set(
            activity.teacher_id for activity in activities_dict.values())}  # Track professor slots

        for activity in activities_dict.values():
            # course = courses[course_id]

            student_groups = activity.group_ids
            rooms = [room for room in spaces_dict.values(
            ) if room.size >= get_classsize(activity)]
            # student_group = course.student  # Get student group
            selected_room = random.choice(rooms)  # Random room

            # Find consecutive slots for the course
            valid_slots = [
                slot for slot in slots if slot not in room_slot_assignments[selected_room.code] and slot not in prof_slot_assignments[activity.teacher_id]]

            if len(valid_slots) >= activity.duration:
                selected_slots = random.sample(valid_slots, activity.duration)
            else:
                selected_slots = random.sample(slots, activity.duration)

            for slot in selected_slots:
                room_slot_assignments[selected_room.code].add(slot)
                prof_slot_assignments[activity.teacher_id].add(slot)

            solution.append(
                (activity.id, student_groups, selected_room.code, selected_slots, activity.teacher_id))
        
        population.append(solution)
        print(evaluator(adapter(solution)))
    return population

# Crossover function


def crossover(parent1, parent2):
    crossover_point = random.choice(range(1, len(parent1)))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2


# NSGA-II selection based on non-dominated sorting and crowding distance
def non_dominated_sorting(population, fitness_values):
    fronts = [[]]
    rank = [0] * len(population)
    domination_count = [0] * len(population)
    dominated_solutions = [[] for _ in range(len(population))]

    for p in range(len(population)):
        for q in range(len(population)):
            if dominates(fitness_values[p], fitness_values[q]):
                dominated_solutions[p].append(q)
            elif dominates(fitness_values[q], fitness_values[p]):
                domination_count[p] += 1

        if domination_count[p] == 0:
            rank[p] = 0
            fronts[0].append(p)

    front_counter = 0
    while len(fronts[front_counter]) > 0:
        next_front = []
        for p in fronts[front_counter]:
            for q in dominated_solutions[p]:
                domination_count[q] -= 1
                if domination_count[q] == 0:
                    rank[q] = front_counter + 1
                    next_front.append(q)
        front_counter += 1
        fronts.append(next_front)

    return fronts


# Crowding distance for maintaining diversity in the population
def crowding_distance(front, fitness_values):
    distance = [0] * len(front)
    if len(front) > 1:  # Only calculate crowding distance if there are at least two individuals in the front
        sorted_by_first = sorted(front, key=lambda x: fitness_values[x][0])
        sorted_by_second = sorted(front, key=lambda x: fitness_values[x][1])

        # Set infinity for boundary individuals
        distance[0] = distance[-1] = float('inf')
        for i in range(1, len(front) - 1):
            distance[i] = (fitness_values[sorted_by_first[i + 1]][0] - fitness_values[sorted_by_first[i - 1]][0]) + \
                          (fitness_values[sorted_by_second[i + 1]][1] -
                           fitness_values[sorted_by_second[i - 1]][1])

    return distance


# Tournament selection for parents
def tournament_selection(population, fitness_values):
    selected_parents = random.sample(list(range(len(population))), 2)
    p1, p2 = selected_parents[0], selected_parents[1]

    if dominates(fitness_values[p1], fitness_values[p2]):
        return p1
    else:
        return p2


def mutate(solution):
    mutation_point = random.randint(0, len(solution) - 1)
    course_id, student_group, _, _, _ = solution[mutation_point]

    new_room = random.choice(list(spaces_dict.values())).code  # Random room
    new_slot = random.choice(slots)  # Random slot

    professor = activities_dict[course_id].teacher_id
    # Update with new room and slot, keep course_id and student_group the same
    solution[mutation_point] = (
        course_id, student_group, new_room, new_slot, professor)
    return solution

# Domination check


def dominates(fit1, fit2):
    return (fit1[0] <= fit2[0] and fit1[1] <= fit2[1] and fit1[2] <= fit2[2]) and \
           (fit1[0] < fit2[0] or fit1[1] < fit2[1] or fit1[2] < fit2[2])


# Run NSGA-II algorithm
def run_nsga2(pop_size=50, generations=1000):
    population = create_population(pop_size)
    for generation in range(generations):
        # Evaluate the fitness of the current population
        fitness_values = [fitness(solution) for solution in population]

        # Non-dominated sorting
        fronts = non_dominated_sorting(population, fitness_values)
        new_population = []

        # Iterate through each front and apply crossover and mutation
        for front in fronts:
            distances = crowding_distance(front, fitness_values)
            sorted_front = sorted(
                front, key=lambda x: distances[front.index(x)], reverse=True)

            # Select parents from the sorted front
            while len(new_population) < pop_size and len(sorted_front) > 1:
                parent1 = population[sorted_front[0]]
                parent2 = population[sorted_front[1]]

                # Crossover
                child1, child2 = crossover(parent1, parent2)

                # Mutation
                child1 = mutate(child1)
                child2 = mutate(child2)

                # Add children to the new population
                new_population.append(child1)
                new_population.append(child2)

        # Truncate to maintain population size
        population = new_population[:pop_size]

        # Print the progress
        if generation % 10 == 0:
            print(
                f"Generation {generation}: Population size {len(population)}")

    return population


# Running the algorithm
best_solution = run_nsga2()
print(f"Best solution: {best_solution}")

 Vacant Rooms : 0 
Lecturer Conflicts : 17
Room Size Conflicts : 0
Sub Group Conflicts:14 
 Vacant Rooms : 0 
Lecturer Conflicts : 13
Room Size Conflicts : 0
Sub Group Conflicts:16 
 Vacant Rooms : 7 
Lecturer Conflicts : 13
Room Size Conflicts : 0
Sub Group Conflicts:9 
 Vacant Rooms : 0 
Lecturer Conflicts : 9
Room Size Conflicts : 0
Sub Group Conflicts:12 
 Vacant Rooms : 0 
Lecturer Conflicts : 18
Room Size Conflicts : 0
Sub Group Conflicts:12 
 Vacant Rooms : 0 
Lecturer Conflicts : 11
Room Size Conflicts : 0
Sub Group Conflicts:15 
 Vacant Rooms : 0 
Lecturer Conflicts : 8
Room Size Conflicts : 0
Sub Group Conflicts:15 
 Vacant Rooms : 6 
Lecturer Conflicts : 19
Room Size Conflicts : 0
Sub Group Conflicts:8 
 Vacant Rooms : 2 
Lecturer Conflicts : 15
Room Size Conflicts : 0
Sub Group Conflicts:20 
 Vacant Rooms : 0 
Lecturer Conflicts : 14
Room Size Conflicts : 0
Sub Group Conflicts:20 
 Vacant Rooms : 0 
Lecturer Conflicts : 11
Room Size Conflicts : 0
Sub Group Conflicts:20 
 Va

In [19]:
evaluator(adapter(best_solution[0]))

' Vacant Rooms : 47 \nLecturer Conflicts : 11\nRoom Size Conflicts : 0\nSub Group Conflicts:6 '

In [5]:
minimum_conflicts = 1000
final_solution = "hello"
for solution in best_solution:
    if minimum_conflicts >= sum(fitness(solution)):
        minimum_conflicts = sum(fitness(solution))
        final_solution = solution
print(minimum_conflicts)

17


In [17]:
# Activity(id=AC-057, subject=IT2010, teacher_id=FA0000010, group_ids=['Y2S1.3'], duration=1),
def adapter(solution):
    timetable = {}
    for slot in slots:
        timetable[slot] = {}
        for space in spaces_dict.keys():
            timetable[slot][space] = ""

    for period in solution:
        time_slots = period[3]
        room = period[2]
        activity_id = period[0]
        if not isinstance(time_slots, list):
            time_slots = [time_slots]
        for aslot in time_slots:
            timetable[aslot][room] = activities_dict[activity_id]
    return timetable

timetable = adapter(final_solution)

In [ ]:
vacant_rooms = []
def evaluator(timetable):
    vacant_room = 0
    prof_conflicts = 0
    room_size_conflicts = 0
    sub_group_conflicts = 0
    unasigned_activities = len(activities_dict)
    activities_set = set()

    for slot in timetable:
        prof_set = set()
        sub_group_set = set()
        for room in timetable[slot]:
            activity = timetable[slot][room]

            if not isinstance(activity, Activity):
                vacant_room += 1
                vacant_rooms.append((slot,room))

            else:
                activities_set.add(activity.id)
                if activity.teacher_id in prof_set:
                    prof_conflicts += 1

                sub_group_conflicts += len(
                    set(activity.group_ids).intersection(sub_group_set))

                group_size = 0
                for group_id in activity.group_ids:
                    group_size += groups_dict[group_id].size
                    sub_group_set.add(group_id)

                if group_size > spaces_dict[room].size:
                    room_size_conflicts += 1
                teacher_id = activity.teacher_id
                prof_set.add(teacher_id)
    unasigned_activities -= len(activities_set)
    return vacant_room, prof_conflicts, room_size_conflicts, sub_group_conflicts, unasigned_activities
    # return f" Vacant Rooms : {vacant_room} \nLecturer Conflicts : {prof_conflicts}\nRoom Size Conflicts : {room_size_conflicts}\nSub Group Conflicts:{sub_group_conflicts} \nNo:of Unassigned Activities:{unasigned_activities} "

In [30]:
evaluator(timetable)

(39, 9, 0, 5, 82)

In [37]:
def print_metric(vacant_rooms, prof_conflicts, room_size_conflicts,sub_group_conflicts, unassigned_activities):
    return f" Vacant Rooms : {vacant_rooms} \nLecturer Conflicts : {prof_conflicts}\nRoom Size Conflicts : {room_size_conflicts}\nSub Group Conflicts:{sub_group_conflicts} \nNo:of Unassigned Activities:{unassigned_activities} "

print_metric(*evaluator(timetable))

' Vacant Rooms : 39 \nLecturer Conflicts : 9\nRoom Size Conflicts : 0\nSub Group Conflicts:5 \nNo:of Unassigned Activities:82 '

In [21]:
timetable

{'MON1': {'LH401': '',
  'LH501': Activity(id=AC-194, subject=IT4570, teacher_id=FA0000005, group_ids=['Y4S2.4'], duration=1),
  'LAB501': Activity(id=AC-170, subject=IT4020, teacher_id=FA0000006, group_ids=['Y4S1.4'], duration=1),
  'LAB502': Activity(id=AC-008, subject=IT1020, teacher_id=FA0000004, group_ids=['Y1S1.1'], duration=1)},
 'MON2': {'LH401': Activity(id=AC-125, subject=IT3030, teacher_id=FA0000006, group_ids=['Y3S1.1'], duration=1),
  'LH501': Activity(id=AC-136, subject=IT3550, teacher_id=FA0000002, group_ids=['Y3S2.1', 'Y3S2.2', 'Y3S2.3', 'Y3S2.4', 'Y3S2.5'], duration=2),
  'LAB501': '',
  'LAB502': ''},
 'MON3': {'LH401': Activity(id=AC-053, subject=IT1580, teacher_id=FA0000001, group_ids=['Y1S2.5'], duration=1),
  'LH501': '',
  'LAB501': Activity(id=AC-045, subject=IT1570, teacher_id=FA0000004, group_ids=['Y1S2.3'], duration=1),
  'LAB502': Activity(id=AC-168, subject=IT4020, teacher_id=FA0000006, group_ids=['Y4S1.2'], duration=1)},
 'MON4': {'LH401': Activity(id=AC-1